In [8]:
import cv2
import numpy as np
import gradio as gr

css = """
.image-container .icon-buttons {display: none !important;}
footer {display: none !important;} 
#component-0 .icon-buttons {display: none !important;}
button.icon {display: none !important;}
"""

def cartoonize_image_logic(img, intensity):
    max_size = 1000
    h, w = img.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        img = cv2.resize(img, (int(w * scale), int(h * scale)))
        
    if img is None:
        return None
    
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    max_size = 800
    h, w = img_bgr.shape[:2]
    if max(h, w) > max_size:
        scale = max_size / max(h, w)
        img_bgr = cv2.resize(img_bgr, (int(w * scale), int(h * scale)))

    filter_iterations = int(intensity / 10) + 1
    sigmaColor = int(intensity * 2) + 10
    
    img_small = cv2.pyrDown(cv2.pyrDown(img_bgr))
    for _ in range(filter_iterations):
        img_small = cv2.bilateralFilter(img_small, d=9, sigmaColor=sigmaColor, sigmaSpace=7)
    
    img_cartoon = cv2.pyrUp(cv2.pyrUp(img_small))
    img_cartoon = cv2.resize(img_cartoon, (img_bgr.shape[1], img_bgr.shape[0]))
    
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.medianBlur(img_gray, 7)
    
    base_block = int(intensity / 5)
    blockSize = base_block if base_block % 2 == 1 else base_block + 1
    if blockSize < 3: blockSize = 3
    
    img_edge = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize, 2)
    img_edge_colored = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2BGR)
    
    cartoon_bgr = cv2.bitwise_and(img_cartoon, img_edge_colored)
    return cv2.cvtColor(cartoon_bgr, cv2.COLOR_BGR2RGB)

with gr.Blocks(css=css) as demo:
    gr.Markdown("# 🎨 影像卡通化轉換器")
    
    with gr.Row():
        with gr.Column():
            input_img = gr.Image(
                label="上傳圖片", 
                sources=["upload"], 
                type="numpy"
            )
            intensity_slider = gr.Slider(
                0, 100, value=50, 
                label="卡通化程度",
            )
        with gr.Column():
            output_img = gr.Image(label="卡通化結果")

    input_img.change(cartoonize_image_logic, [input_img, intensity_slider], output_img)
    intensity_slider.release(cartoonize_image_logic, [input_img, intensity_slider], output_img)

demo.close()
demo.launch()

Running on local URL:  http://127.0.0.1:7904

To create a public link, set `share=True` in `launch()`.
